<div class="alert alert-block alert-info">
<b>Deadline:</b> March 20, 2024 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [32]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [51]:
skip_training = True  # Set this flag to True before validation and submission

In [52]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [53]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is ../data


In [54]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [55]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the item (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5).

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [56]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [57]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [58]:
# architecture( size of the embedding layer, size of the hidden layer, batch size, learning rate,
#               momentum, weight decay, learning rate schedule steps, learning rate schedule factor (gamma),
#               tolerance for early stopping, patience for early stopping)

class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture=(8,64,128,0.05,0.99,0.001,50, 0.5,0.0001,20)
                ):
        
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        super(RecommenderSystem, self).__init__()
        embedding_size = 8 # embedding_size = 10
        hidden_size = 64 # hidden_size = 50
        self.user_embedding = nn.Embedding(n_users + 1, embedding_size)
        self.item_embedding = nn.Embedding(n_items + 1, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        user_embedded = self.user_embedding(user_ids)
        item_embedded = self.item_embedding(item_ids)
        merged = torch.cat([user_embedded, item_embedded], dim=1)
        hidden = self.relu(self.fc1(merged))
        output = self.fc2(hidden)
        return output.squeeze()

You can test the shapes of the model outputs using the function below.

In [59]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [60]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [61]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [62]:
# Helper class(es) and function(s) to regularize the network

class EarlyStopping:
    def __init__(self, tolerance, patience):
        self.tolerance = tolerance
        self.patience = patience
    
    def stop_criterion(self, val_errors):
        if len(val_errors) <= self.patience:
            return False
        
        min_val_error = min(val_errors)
        recent_val_errors = val_errors[-self.patience:]
        return all(error > min_val_error + self.tolerance for error in recent_val_errors)
    

def add_noise(x, noise_std):
    return x + noise_std * torch.randn(x.shape)

In [63]:
# This is the function to print the progress during training
def print_progress(epoch, train_error, val_error):
    print('Epoch {}: Train error: {:.4f}, Test error: {:.4f}'.format(
        epoch, train_error, val_error))

In [64]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device)

if torch.cuda.is_available():
    # Storing ID of current CUDA device
    cuda_id = torch.cuda.current_device()
    print(f"ID of current CUDA device: {torch.cuda.current_device()}")
        
    print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

    print(device.type)

ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce RTX 2080
cpu


In [65]:
# Data loaders
batch_size = 128
train_loader    = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [67]:
model = RecommenderSystem(trainset.n_users, trainset.n_items) # Reset the model for testing
# Implement the training loop in this cell
if not skip_training:
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.99, weight_decay=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)
    epochs = 1000
    model.to(device)

    ids_test = []
    items_test = []
    ratings_test = []

    for i in range(len(testset)):
        user_id, item_id, rating = testset[i]
        ids_test.append(user_id)
        items_test.append(item_id)
        ratings_test.append(rating)

    ids_test = torch.LongTensor(ids_test)
    items_test = torch.LongTensor(items_test)
    ratings_test = torch.LongTensor(ratings_test)

    ids_test = to_device(ids_test, device)
    items_test = to_device(items_test, device)
    ratings_test = to_device(ratings_test, device)

    train_errors = []
    val_errors = []
    early_stop = EarlyStopping(tolerance=0.001, patience=20)
    std1 = 0.5
    std2 = 0.3
    
    for epoch in range(epochs):
        model.train()
        train_losses = []

        scheduler.step()

        for batch in train_loader:
            ids = to_device(batch[0], device)
            items = to_device(batch[1], device)
            ratings = to_device(batch[2], device)

            optimizer.zero_grad()
            outputs = model(ids, items)
            loss = loss_fn(outputs, ratings.float())
            loss.backward()
            optimizer.step()
            

            train_losses.append(loss.item())

        avg_train_loss = sum(train_losses) / len(train_losses)
        train_errors.append(avg_train_loss)

        # Validation error
        model.eval()
        with torch.no_grad():
            outputs_test = model(ids_test, items_test)
            val_loss = loss_fn(outputs_test, ratings_test.float())
            val_errors.append(val_loss.item())

        if early_stop.stop_criterion(val_errors):
            print('Stop after %d epochs' % epoch)
            break

        if epoch % 10 == 0:
            print_progress(epoch, avg_train_loss, val_loss.item())
    
    print('Finished Training')
    
    plt.plot(train_errors, label='train')
    plt.plot(val_errors, label='val')
    plt.legend()
    plt.show()

    print('Final test error:', loss_fn(model(ids_test, items_test), ratings_test.float()).item())

In [68]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=True)

In [69]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

Model loaded from recsys.pth.


The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [70]:
# This cell tests the accuracy of your best model.

In [71]:
# This cell is reserved for grading

In [72]:
# This cell is reserved for grading

In [73]:
# This cell is reserved for grading

In [74]:
# This cell is reserved for grading

In [75]:
# This cell is reserved for grading

In [76]:
# This cell is reserved for grading